In [26]:
import pandas as pd
import nltk
import re
import numpy as np
import time 
import pickle
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

#for RNN
#import tensorflow as tf
#from sklearn.model_selection import train_test_split
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
#import pre-processed data from pickle 
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/sample_results_df_09.pickle")
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/2011_text_wo_names.pickle")
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/2009_text_wo_names.pickle")

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
9,"[leave, band, follow, bust, say, simply, could...",3,0,"[Noel, Gallagher]",True,Noel Gallagher left the Manchester band follow...,5048,2009,0
10,"[launch, clothing, line, earlier, year, admit,...",2,0,[Liam],None,"""Liam launched his clothing line Pretty Green ...",5048,2009,0
11,"[thinking, next, step, musically, mind, say]",1,0,[],None,"""I'm thinking of what the next step is musical...",5048,2009,0
13,"[people, able, buy, record]",1,0,[],None,"""People will be able to buy his records.",5048,2009,0
33,"[come, back, hang, fellow, cast, member, mitch...",0,2,[Sam],None,She's coming back as Sam Mitchell and was hang...,8981,2009,1
...,...,...,...,...,...,...,...,...,...
19327,"[report, speculate, mime, part, track, want, p...",0,2,[],None,Some reports have speculated that she mimed pa...,2157826,2009,1
19328,"[know, dance, lot]",0,1,[],None,"""I know she was dancing a lot.",2157826,2009,1
19329,"[know, mime, think, really, great, performance...",0,2,[],None,I don't know if she was miming or not but I th...,2157826,2009,1
19330,"[legend, make, comeback, show, rendition, new,...",2,0,"[Pop, Robbie]",True,Pop legend Robbie Williams made his comeback o...,2157826,2009,0


In [28]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [36]:
#apply function to only get rows with an absolute count 
df_09['col_type'] = df_09.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_09= df_09[df_09["col_type"].notnull()]

#DOC: number of male and female columns
df_09["col_type"].value_counts()  

0    3339
1    2003
Name: col_type, dtype: int64

**Define TFIDF Vectorizer**

In [30]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [7]:
#word2vec to dos
#import gensim
#from gensim.models import Word2Vec
#file_w2vec= (r"/Users/yolandaferreirofranchi/Desktop/GoogleNews-vectors-negative300.bin") #yolanda's path 
#model_w2v = gensim.models.KeyedVectors.load_word2vec_format(file_w2vec, binary=True)

In [33]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag' 'saga'], 
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:100]
    low_coef = sorted_coef[-100:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])


    #print probability results 
    feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    positive_probas = probas[:, 1]

    print(f"\nProbability of the Following Words Being Female:")
    feature_prob_dict = {}
    for feature, index in feature_indices.items():
        proba = (X_transformed[:, index].toarray() * positive_probas).mean()
        feature_prob_dict[feature] = proba
        print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #save model 
    with open('results.pkl', 'wb') as f:
        pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef, df_probs, best_clf

**Coefficient Analysis**

In [34]:
df_probs_09 = logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')
df_probs_09

Accuracy: 0.91
Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93       657
         1.0       0.87      0.91      0.89       412

    accuracy                           0.91      1069
   macro avg       0.91      0.91      0.91      1069
weighted avg       0.91      0.91      0.91      1069


Probability of the Following Words Being Female:
abandon: 0.0003419629096697936
abdelbaset: 4.635760904151317e-05
abdomen: 7.48839528692512e-05
ability: 8.094783172379096e-05
able: 0.000889024532263488
absence: 0.00012860189171480513
absolute: 3.473340930253683e-05
absolutely: 0.0005210431474188
absolve: 5.700490625218318e-05
absoultely: 5.345273815749968e-05
abuse: 0.0006356062942055645
abusive: 4.984089987653053e-05
accept: 0.0002826795441909251
acceptance: 4.217443563240837e-05
accepting: 5.581951928170968e-05
access: 7.627592664639346e-05
accessible: 4.41469540518553e-05
accident: 0.00010465450654849794
accidental: 6.43163

(      feature       coef
 0        jade  13.845370
 1   boyfriend  12.797718
 2    creative  12.640788
 3        mime  12.602833
 4      french  12.060647
 ..        ...        ...
 95    attract   7.529917
 96  spotlight   7.500673
 97  australia   7.500021
 98     stress   7.488658
 99    airport   7.487627
 
 [100 rows x 2 columns],
      feature       coef
 0   quantity  -6.422150
 1    whether  -6.433374
 2    rubbish  -6.439501
 3      count  -6.464903
 4       desk  -6.481465
 ..       ...        ...
 95  formerly -10.438487
 96   country -10.932315
 97     blues -11.131853
 98    bottom -12.641673
 99      neil -13.356755
 
 [100 rows x 2 columns],
             probability
 abandon        0.000342
 abdelbaset     0.000046
 abdomen        0.000075
 ability        0.000081
 able           0.000889
 ...                 ...
 zane           0.000118
 zay            0.000108
 zee            0.000108
 zone           0.000061
 zoom           0.000035
 
 [4346 rows x 1 columns],
 Logis

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [44]:
#open model performance metrics 
with open('results_09.pkl', 'rb') as f:
    results = pickle.load(f)

accuracy = results['accuracy']
report = results['report']

print(f"Accuracy: {accuracy}")
print(f"Classification report:\n{report}")

Accuracy: 0.9130028063610851
Classification report:
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93       657
         1.0       0.87      0.91      0.89       412

    accuracy                           0.91      1069
   macro avg       0.91      0.91      0.91      1069
weighted avg       0.91      0.91      0.91      1069



In [43]:
#create DF of highest coef
highest_coef_09 = pd.DataFrame(df_probs_09[0]) 

#create DF of lowest lowest coef manipulation 
lowest_coef_09 = pd.DataFrame(df_probs_09[1]) 
lowest_coef_09 = lowest_coef_09.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_09.head(20)

,feature,coef
0,neil,-13.356755
1,bottom,-12.641673
2,blues,-11.131853
3,country,-10.932315
4,formerly,-10.438487
5,liam,-10.293463
6,european,-10.203249
7,suppose,-10.024999
8,surreal,-10.013806
9,adventure,-9.816442


In [46]:
#save DF as pickle file per year 
lowest_coef_09.to_pickle('RESULTS09_coef_low.pickle')
highest_coef_09.to_pickle('RESULTS09_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Word Probability Analysis** 

#make sure to un-comment the last two lines of the functions

In [61]:
#access list of top words 
word_prob_09 = pd.DataFrame(df_probs_09[2]) 
word_prob_09 = word_prob_09.sort_values(["probability"], ascending= False)

#save list of words (highest to lowest probability of being female) for year: 
word_prob_09.to_pickle('RESULTS09_words_prob.pickle')
word_prob_09.head(10)

,probability
say,0.018780
get,0.008789
year,0.006995
go,0.006874
think,0.006431
like,0.006220
really,0.005678
work,0.005391
would,0.004943
album,0.004710


**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

**Support Vector Machine**

In [19]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
def svm_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'C': [.001, .01, .1, 1, 10, 100],
        #'kernel': default('rbf')',
        'degree': [2,3,4],
        'gamma': ['scale', 'auto'],
        #'class_weight': [None],
        #'random_state': [42]
    }

    #the classifier 
    clf = SVC()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = SVC(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Year: {year}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #get important features (equivalent of coefficients in logreg)
    result_clf = permutation_importance(best_clf, X_test, y_test, n_repeats=10, random_state=42)
    importance_scores = result_clf.importances_mean
    importance_scores_df = pd.DataFrame(importance_scores, columns=['tbd'])

    for i in range(len(importance_scores)):
        print("Feature {}: Importance score = {:.3f}".format(i, importance_scores[i]))
    
    #print probability results 
    #feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    #probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    #positive_probas = probas[:, 1]

    #print(f"\nProbability of the Following Words Being Female:")
    #feature_prob_dict = {}
    #for feature, index in feature_indices.items():
        #proba = (X_transformed[:, index] * positive_probas).mean()
        #feature_prob_dict[feature] = proba
        #print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return importance_scores_df

In [20]:
svm_year(df_09, 'col_type', 'pre_processed_sent') #takes more than 1 hour to run

Accuracy: 0.70
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.88      0.78       418
           1       0.66      0.38      0.48       248

    accuracy                           0.70       666
   macro avg       0.68      0.63      0.63       666
weighted avg       0.69      0.70      0.67       666



KeyboardInterrupt: 

In [ ]:
#save the SVM model 
import joblib
model, accuracy, class_report = svm_year(df_09, 'col_type', 'pre_processed_sent')

# Save the model and performance metrics
joblib.dump(model, 'model_svm.pkl')
with open('performance_metrics.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Classification Report:\n{class_report}")

**Neural Network** 

In [30]:
def RNN_lstm(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    # Preprocess text
    tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post', padding='post')
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)
    
    # Define model
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(10000, 32),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    
    # Evaluate model on test set
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

**Comparing the Models**

In [25]:
model_results = {' ': ['Logistic Regression', 'SVM', 'NN'],
           'Accuracy': [0.66, 0.70, 0],
           'Time (s)': [54.39, 3540, 0]}

# Create a pandas dataframe from the dictionary
df_mod_results = pd.DataFrame(model_results)

# Set the index of the dataframe to the Kernel column
df_mod_results.set_index(' ', inplace=True)

# Display the dataframe
print(df_mod_results)

                     Accuracy  Time (s)
                                       
Logistic Regression      0.66     54.39
SVM                      0.70   3540.00
NN                       0.00      0.00
